In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

import pandas as pd
import os
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from model_trainer.entities import ANOMALY_ARCHIVE_ENTITIES, MACHINES, MSL_CHANNELS, SMAP_CHANNELS, ANOMALY_ARCHIVE_ENTITY_TO_DATA_FAMILY
from metrics.metrics import METRICS_NAMES, evaluate_model_selection
from model_selection.rank_aggregation import trimmed_borda, trimmed_kemeny, borda, kemeny

from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import ndcg_score
from tqdm import tqdm 
from mallows import mallows_kendall as mk
from datetime import datetime
from pprint import pprint

In [ ]:
SAVE_DIR = '/home/ubuntu/efs/results'

DATASETS = ['anomaly_archive', 'smd', 'msl', 'smap']
ENTITIES = [ANOMALY_ARCHIVE_ENTITIES, MACHINES, MSL_CHANNELS, SMAP_CHANNELS]
DATASET_ENTITY = dict(zip(DATASETS, ENTITIES))

In [ ]:
dataset = 'smd'

cd_stats = []
for entity in tqdm(DATASET_ENTITY[dataset]): 
    ranking_obj_file = f'ranking_obj_{entity}.data'
    try:
        with open(os.path.join(SAVE_DIR, dataset, ranking_obj_file), 'rb') as f: 
            ranking_obj = pkl.load(f)
        
        ranks = ranking_obj.ranks_by_metrics.astype(int)
        rank_prauc = ranking_obj.rank_prauc.astype(int)
        rank_f1 = ranking_obj.rank_f1.astype(int)

    except: 
        continue
    
    if ranks.shape[1] != 19:
        print(entity, ranks.shape[1])
        continue

    # Get metric names
    metric_names = list(ranking_obj.models_performance_matrix.columns)
    metric_names.remove('PR-AUC')
    metric_names.remove('Best F-1')

    average_dist_of_ranks = squareform(pdist(ranks, metric=mk.distance)).mean(axis=1)
    clustering = AgglomerativeClustering(n_clusters=2, linkage='single').fit_predict(average_dist_of_ranks.reshape((-1, 1)))
    cluster_ids, counts = np.unique(clustering, return_counts=True)
    max_cluster = cluster_ids[np.argmax(counts)]
    _, trimmed_kemeny_rank = kemeny(ranks[np.where(clustering == max_cluster)[0], :], verbose=False)
    trimmed_kemeny_rank = trimmed_kemeny_rank.astype(int)

    _, kemeny_rank = kemeny(ranks, verbose=False)
    kemeny_rank = kemeny_rank.astype(int)

    praucs = ranking_obj.models_performance_matrix.iloc[:, 0].to_numpy().squeeze()
    f1s = ranking_obj.models_performance_matrix.iloc[:, 1].to_numpy().squeeze()
    model_names = np.array(list(ranking_obj.models_performance_matrix.index))
    
    if np.std(praucs) == 0.0: # Bad datasets have 0.5 prauc for all models
        continue

    # if max(praucs) < 0.5: # Bad models
    #     continue
    
    for i, mn in enumerate(model_names): 
        cd_stats.append([entity, 
                         mn, 
                         ranking_obj.models_performance_matrix.iloc[i, 0], 
                         ranking_obj.models_performance_matrix.iloc[i, 1]])
    cd_stats.append([entity, 'Trimmed Aggregation', 
                     praucs[trimmed_kemeny_rank[0]], 
                     f1s[trimmed_kemeny_rank[0]]])
    cd_stats.append([entity, 'Untrimmed Aggregation', 
                     praucs[kemeny_rank[0]], 
                     f1s[kemeny_rank[0]]])
    cd_stats.append([entity, 'Best Model', np.max(praucs), np.max(praucs)])
    # cd_stats.append([entity, 'Randomly Chosen Model', np.mean(praucs), np.mean(praucs)])

In [ ]:
cd_stats = pd.DataFrame(np.array(cd_stats), columns=['dataset_name', 'classifier_name', 'prauc', 'f1'], index=None)
cd_stats

In [ ]:
perf_data = cd_stats.loc[:, ['dataset_name', 'classifier_name', 'prauc']]
perf_data = perf_data.rename(columns={"prauc": "accuracy"})

In [ ]:
from data.cd_diagram import draw_cd_diagram
draw_cd_diagram(df_perf=perf_data, title='PR-AUC', labels=True)